In [ ]:
import ipywidgets as widgets

wordlist_textarea = widgets.Textarea(description="Vocabulary:", layout=widgets.Layout(height="100px"))
n_words_slider = widgets.IntSlider(value=50, min=30, max=100, step=10, description="Number of output words:")
n_words_slider.style.description_width = "50%"

generate_button = widgets.Button(description="Generate prompt")
clear_button = widgets.Button(description="Clear")
buttons_hbox = widgets.HBox([generate_button, clear_button])

prompt_textarea = widgets.Textarea(description="Prompt:", layout=widgets.Layout(height="160px"))

input_widgets_vbox = widgets.VBox([wordlist_textarea, n_words_slider, buttons_hbox])

toplevel_container = widgets.HBox([input_widgets_vbox, prompt_textarea])

def on_generate_prompt(_):
    words = wordlist_textarea.value.split()
    n_output_words = n_words_slider.value
    print(f"{words} | {n_output_words}")

def on_clear(_):
    wordlist_textarea.value = ""

generate_button.on_click(on_generate_prompt)
clear_button.on_click(on_clear)

display(toplevel_container)